In [11]:
import numpy as np
import torch
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
import LoadData as LD

In [12]:
DataArray = LD.GetData()

print( DataArray )

None


In [13]:
mu1 = 0 
sigma1 = 0.5 

dist1 = np.array(  np.random.normal(0, sigma1, 20 ) ,  dtype=np.float32) 
dist2 = np.array( np.random.normal(0.6, sigma1, 20) ,  dtype=np.float32) 
dist3 = np.array( np.random.normal(0.8, sigma1, 20) ,  dtype=np.float32) 
dist4 = np.array( np.random.normal(1.0, sigma1, 20) ,  dtype=np.float32) 

dist5 = np.array( np.random.normal(1.3, sigma1, 20) ,  dtype=np.float32) 
dist6 =  np.array( np.random.normal(2.5, sigma1, 20) ,  dtype=np.float32) 
dist7 = np.array( np.random.normal(5, sigma1, 20) ,  dtype=np.float32) 
dist8 =  np.array( np.random.normal(6, sigma1, 20) ,  dtype=np.float32) 
dist9 =np.array( np.random.normal(5.5, sigma1, 20) ,  dtype=np.float32) 
dist10 = np.array( np.random.normal(8.2, sigma1, 20) ,  dtype=np.float32) 

dist11 = dist1
dist12 = dist3
dist13 = dist6
dist14 = dist8
dist15 = dist9 
dist16 = dist1 
dist17 = dist8


Data1 = np.asarray( [ dist1 , dist2 , dist3 , dist4 , dist5 ,
dist6, dist7 , dist8 , dist9 , dist10 , dist11 , dist12 , dist13 , dist14 , dist15 ,dist16 , dist17 ])

Data1 = torch.tensor(Data1 , device='cpu' , dtype=torch.float32)
print( Data1.shape)
y = torch.tensor([ 1,1,1,1,0,0,0,0,0,0,     1 , 1 ,0,0,0 ,   1 , 0])

torch.Size([17, 20])


In [14]:
edge_index = torch.tensor([[0, 10],
                           [10, 0],

                           [1, 10],
                           [10, 1] ,

                           [2 , 11] , 
                           [11, 2] , 

                           [3 , 11] , 
                           [11 , 3] , 

                           [4 , 12] , 
                           [12 , 4] ,

                           [5,12],
                           [12,5],

                           [6,13],
                           [13,6],

                           [7 , 13],
                           [13 , 7],

                           [8,14],
                           [14,8],

                           [9 ,14],
                           [14,8],

                           [10,15],
                           [15,10],

                           [15,11],
                           [11,15],

                           [12,16],
                           [16,12],

                           [13,16],
                           [16,13],

                           [14,16],
                           [16,14]

                            ] , dtype=torch.long)


edge_index = torch.transpose( edge_index ,  0 , 1)

train_mask = torch.tensor( [ True, True , True, False,True,True,True,True,True,False,False,False,False,False,False,False,False])

data = Data(x=Data1, edge_index=edge_index , y=y , train_mask=train_mask , val_mask= train_mask, test_mask=train_mask)

print(data)

Data(x=[17, 20], edge_index=[2, 30], y=[17], train_mask=[17], val_mask=[17], test_mask=[17])


In [15]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(20, 16)
        self.conv2 = GCNConv(16, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [19]:
device = 'cpu' #torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = GCN().to(device)
# data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(20):
    optimizer.zero_grad()
    out = model(data)
    print(data.y)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


cpu
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0

In [6]:
model.eval()
pred = model(data).argmax(dim=1)
print(pred)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
print(correct)

acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')
torch.save(model, 'collision_detection_GNN.pt')

tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor(8)
Accuracy: 1.0000


In [7]:
model2 = torch.load('collision_detection_GNN.pt')

In [8]:
print(y)
print(data.test_mask)

tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])
tensor([ True,  True,  True, False,  True,  True,  True,  True,  True, False,
        False, False, False, False, False, False, False])


In [9]:
import numpy as np
Adj = np.zeros( ( 17, 17) )

Adj[0][10] = 1
Adj[10][0] = 1


Adj[1][10] = 1
Adj[10][1] = 1

Adj[2][11] = 1
Adj[11][2] = 1

Adj[3][11] = 1
Adj[11][3] = 1

Adj[4][12] = 1
Adj[12][4] = 1

Adj[5][12] = 1
Adj[12][5] = 1

Adj[6][13] = 1
Adj[13][6] = 1


Adj[7][13] = 1
Adj[13][7] = 1


Adj[8][14] = 1
Adj[14][8] = 1

Adj[10][14] = 1
Adj[14][10] = 1

Adj[10][15] = 1
Adj[15][10] = 1

Adj[11][15] = 1
Adj[15][11] = 1


Adj[12][16] = 1
Adj[16][12] = 1

Adj[13][16] = 1
Adj[16][13] = 1


Adj[14][16] = 1
Adj[16][14] = 1

